#**Assessment-1**

In [0]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window as W
from pyspark.sql.types import *
from delta.tables import DeltaTable

In [0]:
spark = SparkSession.builder.appName("dbshell-01").getOrCreate()

# **Data Ingesiton & Schema Analysis**

In [0]:
# 1.1 Load CSV using PySpark with schema inference
dfTrf = spark.read.csv("/FileStore/tables/traffic_logs.csv", header=True, inferSchema=True)

In [0]:
# 1.2 Manually define schema and compare
schema = StructType(
    [
        StructField("LogID", StringType(), nullable=False),
        StructField("VechileID", StringType(), True),
        StructField("EntryPoint", StringType(), True),
        StructField("ExitPoint", StringType(), True),
        StructField("EntryTime", TimestampType(), True),
        StructField("ExitTime", TimestampType(), True),
        StructField("VechileType", StringType(), True),
        StructField("SpeedKMH", FloatType(), True),
        StructField("TollPaid", FloatType(), True)
    ]
)
dfTrfManual = spark.read.csv("/FileStore/tables/traffic_logs.csv", header=True, schema=schema)

print("Infered Schema")
dfTrf.printSchema()

print("Manual Schema")
dfTrfManual.printSchema()

Infered Schema
root
 |-- LogID: string (nullable = true)
 |-- VehicleID: string (nullable = true)
 |-- EntryPoint: string (nullable = true)
 |-- ExitPoint: string (nullable = true)
 |-- EntryTime: timestamp (nullable = true)
 |-- ExitTime: timestamp (nullable = true)
 |-- VehicleType: string (nullable = true)
 |-- SpeedKMH: integer (nullable = true)
 |-- TollPaid: integer (nullable = true)

Manual Schema
root
 |-- LogID: string (nullable = true)
 |-- VechileID: string (nullable = true)
 |-- EntryPoint: string (nullable = true)
 |-- ExitPoint: string (nullable = true)
 |-- EntryTime: timestamp (nullable = true)
 |-- ExitTime: timestamp (nullable = true)
 |-- VechileType: string (nullable = true)
 |-- SpeedKMH: float (nullable = true)
 |-- TollPaid: float (nullable = true)



In [0]:
# 1.3 Ensure EntryTime/ExitTime are timestamp
dfTrf = dfTrf.withColumn("EntryTime", F.to_timestamp(F.col("EntryTime"))) \
             .withColumn("ExitTime", F.to_timestamp(F.col("ExitTime")))
dfTrf.show()

+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
|LogID|VehicleID|EntryPoint|ExitPoint|          EntryTime|           ExitTime|VehicleType|SpeedKMH|TollPaid|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+
| L001|     V001|     GateA|    GateC|2024-05-01 08:01:00|2024-05-01 08:20:00|        Car|      60|      50|
| L002|     V002|     GateB|    GateC|2024-05-01 08:10:00|2024-05-01 08:45:00|      Truck|      45|     100|
| L003|     V003|     GateA|    GateD|2024-05-01 09:00:00|2024-05-01 09:18:00|       Bike|      55|      30|
| L004|     V004|     GateC|    GateD|2024-05-01 09:15:00|2024-05-01 09:35:00|        Car|      80|      50|
| L005|     V005|     GateB|    GateA|2024-05-01 10:05:00|2024-05-01 10:40:00|        Bus|      40|      70|
+-----+---------+----------+---------+-------------------+-------------------+-----------+--------+--------+



# **Derived Column Creation**

In [0]:
# 2.1 Calculate TripDurationMinutes = ExitTime - EntryTime
dfTrf = dfTrf.withColumn("TripDurationMinutes", (F.unix_timestamp(dfTrf.ExitTime) - F.unix_timestamp(dfTrf.EntryTime)) / 60) 
dfTrf.select(["VehicleID","VehicleType" , "TripDurationMinutes"]).show()

+---------+-----------+-------------------+
|VehicleID|VehicleType|TripDurationMinutes|
+---------+-----------+-------------------+
|     V001|        Car|               19.0|
|     V002|      Truck|               35.0|
|     V003|       Bike|               18.0|
|     V004|        Car|               20.0|
|     V005|        Bus|               35.0|
+---------+-----------+-------------------+



In [0]:
# 2.2 Add IsOverspeed = SpeedKMH > 60
dfTrf.withColumn("IsOverSpeed", F.when(F.col("SpeedKMH") > 60, 1).otherwise(0)) \
    .select(["VehicleID","VehicleType" ,"SpeedKMH" , "IsOverSpeed"]) \
    .show()

+---------+-----------+--------+-----------+
|VehicleID|VehicleType|SpeedKMH|IsOverSpeed|
+---------+-----------+--------+-----------+
|     V001|        Car|      60|          0|
|     V002|      Truck|      45|          0|
|     V003|       Bike|      55|          0|
|     V004|        Car|      80|          1|
|     V005|        Bus|      40|          0|
+---------+-----------+--------+-----------+



# **Vehicle Behavior Aggregations**

In [0]:
# 3.1 Average speed per VehicleType
dfTrf.groupBy("VehicleType") \
  .agg(
    F.round(F.mean("SpeedKMH"), 2).alias("AverageSpeed")
  ) \
  .show()

+-----------+------------+
|VehicleType|AverageSpeed|
+-----------+------------+
|       Bike|        55.0|
|        Car|        70.0|
|      Truck|        45.0|
|        Bus|        40.0|
+-----------+------------+



In [0]:
# 3.2 Total toll collected per gate (EntryPoint)
dfTrf.groupBy("EntryPoint") \
    .agg(
        F.sum("TollPaid").alias("TotalAmount")
    ) \
    .show()

+----------+-----------+
|EntryPoint|TotalAmount|
+----------+-----------+
|     GateA|         80|
|     GateB|        170|
|     GateC|         50|
+----------+-----------+



In [0]:
# 3.3 Most used ExitPoint
dfTrf.groupBy("ExitPoint") \
    .agg(
        F.count("ExitPoint").alias("UsageCount")
    ) \
    .sort("UsageCount", ascending=False) \
    .show(1)

+---------+----------+
|ExitPoint|UsageCount|
+---------+----------+
|    GateD|         2|
+---------+----------+
only showing top 1 row



# **Window Functions**

In [0]:
# 4.1 Rank vehicles by speed within VehicleType
win = W.partitionBy("VehicleType").orderBy(F.desc("SpeedKMH"))
dfTrf.withColumn("SpeedRank", F.rank().over(win)) \
    .select(["VehicleID","VehicleType" ,"SpeedKMH" , "SpeedRank"]) \
    .show()

+---------+-----------+--------+---------+
|VehicleID|VehicleType|SpeedKMH|SpeedRank|
+---------+-----------+--------+---------+
|     V003|       Bike|      55|        1|
|     V005|        Bus|      40|        1|
|     V004|        Car|      80|        1|
|     V001|        Car|      60|        2|
|     V002|      Truck|      45|        1|
+---------+-----------+--------+---------+



In [0]:
# 4.2 Find last exit time for each vehicle using lag()
win2 = W.orderBy("ExitTime")
dfTrf.withColumn("LastExitTime", F.lag("ExitTime", 1).over(win2)) \
    .select(["VehicleID","VehicleType" ,"ExitTime" , "LastExitTime"]) \
    .show()

+---------+-----------+-------------------+-------------------+
|VehicleID|VehicleType|           ExitTime|       LastExitTime|
+---------+-----------+-------------------+-------------------+
|     V001|        Car|2024-05-01 08:20:00|               null|
|     V002|      Truck|2024-05-01 08:45:00|2024-05-01 08:20:00|
|     V003|       Bike|2024-05-01 09:18:00|2024-05-01 08:45:00|
|     V004|        Car|2024-05-01 09:35:00|2024-05-01 09:18:00|
|     V005|        Bus|2024-05-01 10:40:00|2024-05-01 09:35:00|
+---------+-----------+-------------------+-------------------+



# **Session Segmentation**

In [0]:
# 5.1 Group by VehicleID to simulate route sessions
dfTrf.groupBy("VehicleID") \
  .agg(
    F.countDistinct("EntryPoint").alias("DifferentEntryPoints"),
    F.countDistinct("ExitPoint").alias("DifferentExitPoints"),
    F.sum("TripDurationMinutes").alias("TotalMinutes"),
    F.round(F.mean("SpeedKMH"), 2).alias("AverageSpeed"), 
    F.sum("TollPaid").alias("TotalTollPaid")
  ) \
  .show()

+---------+--------------------+-------------------+------------+------------+-------------+
|VehicleID|DifferentEntryPoints|DifferentExitPoints|TotalMinutes|AverageSpeed|TotalTollPaid|
+---------+--------------------+-------------------+------------+------------+-------------+
|     V004|                   1|                  1|        20.0|        80.0|           50|
|     V005|                   1|                  1|        35.0|        40.0|           70|
|     V001|                   1|                  1|        19.0|        60.0|           50|
|     V003|                   1|                  1|        18.0|        55.0|           30|
|     V002|                   1|                  1|        35.0|        45.0|          100|
+---------+--------------------+-------------------+------------+------------+-------------+



In [0]:
# 5.2 Find duration between subsequent trips (idle time)
win3 = W.partitionBy("VehicleID").orderBy("EntryTime")

dfTrf.withColumn("NextEntryTime", F.lead("EntryTime", 1).over(win3)) \
    .withColumn("IdleTime", (F.unix_timestamp("NextEntryTime") - F.unix_timestamp("ExitTime")) / 60) \
    .select(["VehicleID", "EntryTime", "ExitTime", "NextEntryTime", "IdleTime"]) \
    .show()

+---------+-------------------+-------------------+-------------+--------+
|VehicleID|          EntryTime|           ExitTime|NextEntryTime|IdleTime|
+---------+-------------------+-------------------+-------------+--------+
|     V001|2024-05-01 08:01:00|2024-05-01 08:20:00|         null|    null|
|     V002|2024-05-01 08:10:00|2024-05-01 08:45:00|         null|    null|
|     V003|2024-05-01 09:00:00|2024-05-01 09:18:00|         null|    null|
|     V004|2024-05-01 09:15:00|2024-05-01 09:35:00|         null|    null|
|     V005|2024-05-01 10:05:00|2024-05-01 10:40:00|         null|    null|
+---------+-------------------+-------------------+-------------+--------+



# **Anomaly Detection**

In [0]:
# 6.1 Identify vehicles with speed > 70 and TripDuration < 10 minutes
dfTrf.filter((dfTrf.SpeedKMH > 70) & (dfTrf.TripDurationMinutes < 10)) \
  .select(["VehicleID", "VehicleType", "SpeedKMH", "TripDurationMinutes"]) \
  .show()

+---------+-----------+--------+-------------------+
|VehicleID|VehicleType|SpeedKMH|TripDurationMinutes|
+---------+-----------+--------+-------------------+
+---------+-----------+--------+-------------------+



In [0]:
# 6.2 Vehicles that paid less toll for longer trips
dfTrf.withColumn("TollPerMinute", F.round(dfTrf.TollPaid / dfTrf.TripDurationMinutes, 2)) \
    .sort(["TripDurationMinutes", "TollPerMinute"], ascending=[False, True]) \
    .select(["VehicleID", "VehicleType", "TripDurationMinutes", "TollPerMinute"]) \
    .show(3)

+---------+-----------+-------------------+-------------+
|VehicleID|VehicleType|TripDurationMinutes|TollPerMinute|
+---------+-----------+-------------------+-------------+
|     V005|        Bus|               35.0|          2.0|
|     V002|      Truck|               35.0|         2.86|
|     V004|        Car|               20.0|          2.5|
+---------+-----------+-------------------+-------------+
only showing top 3 rows



In [0]:
# 6.3 Suspicious backtracking (ExitPoint earlier than EntryPoint)
dfTrf.withColumn("Suspicious", F.when(dfTrf.ExitTime < dfTrf.EntryTime, "Yes").otherwise("No")) \
    .select(["VehicleID", "VehicleType", "Suspicious"]) \
    .show()

+---------+-----------+----------+
|VehicleID|VehicleType|Suspicious|
+---------+-----------+----------+
|     V001|        Car|        No|
|     V002|      Truck|        No|
|     V003|       Bike|        No|
|     V004|        Car|        No|
|     V005|        Bus|        No|
+---------+-----------+----------+



# **Join with Metadata**

In [0]:

# 7.1 Prepare this small vehicle_registry.csv :
data = [
  ("V001","Anil","Hyundai i20","Delhi"),
  ("V002","Rakesh","Tata Truck","Chennai"),
  ("V003","Sana","Yamaha R15","Mumbai"),
  ("V004","Neha","Honda City","Bangalore"),
  ("V005","Zoya","Volvo Bus","Pune")
]
colums = ["VehicleID","OwnerName","Model","RegisteredCity"]

vehicle_registry = spark.createDataFrame(data, colums)
vehicle_registry.show()

+---------+---------+-----------+--------------+
|VehicleID|OwnerName|      Model|RegisteredCity|
+---------+---------+-----------+--------------+
|     V001|     Anil|Hyundai i20|         Delhi|
|     V002|   Rakesh| Tata Truck|       Chennai|
|     V003|     Sana| Yamaha R15|        Mumbai|
|     V004|     Neha| Honda City|     Bangalore|
|     V005|     Zoya|  Volvo Bus|          Pune|
+---------+---------+-----------+--------------+



In [0]:
# 7.2 Join and group trips by RegisteredCity
dfJoined = dfTrf.join(vehicle_registry, on="VehicleID", how="inner")
dfJoined.groupBy("RegisteredCity") \
    .agg(
        F.sum("TripDurationMinutes").alias("TotalTripDuration")
    ) \
    .show()

+--------------+-----------------+
|RegisteredCity|TotalTripDuration|
+--------------+-----------------+
|         Delhi|             19.0|
|       Chennai|             35.0|
|        Mumbai|             18.0|
|     Bangalore|             20.0|
|          Pune|             35.0|
+--------------+-----------------+



# **Delta Lake Features**

In [0]:
# 8.1 Save traffic_logs as Delta Table
spark.sql("CREATE DATABASE IF NOT EXISTS vehicle")
spark.sql("USE vehicle")

dfTrf.write.format("delta").mode("overwrite").saveAsTable("vehicle.traffic_logs")

In [0]:
# 8.2 Apply MERGE INTO to update toll rates for all Bikes
deltaTraffic = DeltaTable.forName(spark, "vehicle.traffic_logs")

bikeupdate = deltaTraffic.toDF() \
             .filter(F.col("VehicleType") == "Bike") \
             .withColumn("TollPaid", F.col("TollPaid") + 50)

bikeupdate.createOrReplaceTempView("bike_update")

spark.sql("""
          MERGE INTO traffic_logs AS target
          USING bike_update AS sourced
          ON target.VehicleID = sourced.VehicleID
          WHEN MATCHED THEN UPDATE SET *
          WHEN NOT MATCHED THEN INSERT *
          """)

Out[62]: DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
# 8.3 Delete trips longer than 60 minutes
spark.sql("""
          DELETE FROM traffic_logs
          WHERE TripDurationMinutes > 60
          """)

Out[66]: DataFrame[num_affected_rows: bigint]

In [0]:
# 8.4 Use DESCRIBE HISTORY and VERSION AS OF
spark.sql("""
        DESCRIBE HISTORY traffic_logs
          """).show()

spark.sql("""
          SELECT * FROM traffic_logs VERSION AS OF 5
          """).show()

+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|           operation| operationParameters| job|          notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      6|2025-06-19 05:24:06|8065524845430037|tharunaadhi6@gmai...|              DELETE|{predicate -> ["(...|null|{4380051481415446}|0619-035229-6g9p739z|          5|WriteSerializable|        false|{numRemovedFiles ...|        null|Databr

# **Advanced Conditions**

In [0]:
#9.1 Tag trip type as:
dfTrf.withColumn("TripType", F.when(dfTrf.TripDurationMinutes < 15, "Short").when(dfTrf.TripDurationMinutes > 30, "Long").otherwise("Medium")) \
    .select(["VehicleID", "TripDurationMinutes", "TripType"]) \
    .show()

+---------+-------------------+--------+
|VehicleID|TripDurationMinutes|TripType|
+---------+-------------------+--------+
|     V001|               19.0|  Medium|
|     V002|               35.0|    Long|
|     V003|               18.0|  Medium|
|     V004|               20.0|  Medium|
|     V005|               35.0|    Long|
+---------+-------------------+--------+



In [0]:
# 9.2 Flag vehicles with more than 3 trips in a day
dfTrf.groupBy("VehicleID").agg(
    F.when(F.count("*") > 3, "Flagged3").otherwise("NotFlagged").alias("Flag")
).show()

+---------+----------+
|VehicleID|      Flag|
+---------+----------+
|     V004|NotFlagged|
|     V005|NotFlagged|
|     V001|NotFlagged|
|     V003|NotFlagged|
|     V002|NotFlagged|
+---------+----------+



# **Export & Reporting**

In [0]:
10.1 # Write final enriched DataFrame to:
# Parquet partitioned by VehicleType
# CSV for dashboards
dfJoined.write.mode("overwrite").csv("/Users/tharunaadhi6@gmail.com/delta tables/dfJoinedcsv")

dfJoined.write.mode("overwrite").parquet("/Users/tharunaadhi6@gmail.com/delta tables/dfJoinedparquet", partitionBy="VehicleType")

In [0]:
# 10.2 Create summary SQL View: total toll by VehicleType + ExitPoint
view = dfTrf.groupBy("VehicleType", "ExitPoint").agg(
    F.sum("TollPaid").alias("TotalToll")
)
view.createOrReplaceTempView("toll_view")

spark.sql("SELECT * FROM toll_view").show()

+-----------+---------+---------+
|VehicleType|ExitPoint|TotalToll|
+-----------+---------+---------+
|        Car|    GateD|       50|
|      Truck|    GateC|      100|
|       Bike|    GateD|       30|
|        Bus|    GateA|       70|
|        Car|    GateC|       50|
+-----------+---------+---------+

